In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

print("✅ Đã khai báo thư viện thành công!")

✅ Đã khai báo thư viện thành công!


In [3]:
# --- 1. TẢI DỮ LIỆU ---
try:
    df = pd.read_csv('../data/processed/clean_vn_housing.csv')
    print("✅ Đã tải dữ liệu thành công!")
    print(f"Kích thước dữ liệu gốc: {df.shape}")
except FileNotFoundError:
    print("❌ Lỗi: Không tìm thấy file 'clean_vn_housing.csv'")

✅ Đã tải dữ liệu thành công!
Kích thước dữ liệu gốc: (36128, 9)


In [4]:
# --- 2. CHUẨN BỊ DỮ LIỆU ---

# Khai báo lại danh sách biến
target = 'Total_Price_Billion'
features = ['District', 'House_type', 'Legal', 'Floors', 'Bedrooms', 'Area']
numeric_features = ['Floors', 'Bedrooms', 'Area']
categorical_features = ['District', 'House_type', 'Legal']

# --- ÉP KIỂU DỮ LIỆU ---
for col in categorical_features:
    df[col] = df[col].astype(str)

# Chọn X, y
X = df[features]
y = df[target]

# Chia tập Train (80%) - Test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Kích thước tập Train: {X_train.shape}")
print(f"Kích thước tập Test: {X_test.shape}")

# Xây dựng Pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

print("✅ Đã xử lý xong!")

Kích thước tập Train: (28902, 6)
Kích thước tập Test: (7226, 6)
✅ Đã xử lý xong!


In [5]:
# --- 3. HUẤN LUYỆN MODEL ---

# Khai báo các mô hình Baseline cần chạy
models = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(alpha=1.0),
    'Lasso': Lasso(alpha=0.01)
}

trained_models = {} # Lưu lại mô hình đã train để dùng sau này nếu cần

print("⏳ Đang huấn luyện các mô hình...")

for name, model in models.items():
    # Tạo pipeline: Xử lý dữ liệu -> Model
    clf = Pipeline(steps=[('preprocessor', preprocessor),
                          ('model', model)])

    # Huấn luyện trên tập Train
    clf.fit(X_train, y_train)

    # Lưu vào danh sách
    trained_models[name] = clf
    print(f"   -> Đã huấn luyện xong: {name}")

print("✅ Hoàn tất huấn luyện!")

⏳ Đang huấn luyện các mô hình...
   -> Đã huấn luyện xong: Linear Regression
   -> Đã huấn luyện xong: Ridge
   -> Đã huấn luyện xong: Lasso
✅ Hoàn tất huấn luyện!


In [6]:
# --- 4. ĐÁNH GIÁ KẾT QUẢ ---

results = []

print("\n--- BẢNG KẾT QUẢ BASELINE (MỐC CHUẨN) ---")
print(f"{'Model':<20} | {'MAE':<10} | {'RMSE':<10} | {'R2 Score':<10}")
print("-" * 60)

for name, clf in trained_models.items():
    # Dự đoán trên tập Test
    y_pred = clf.predict(X_test)

    # Tính toán chỉ số
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    # Lưu kết quả
    results.append({'Model': name, 'MAE': mae, 'RMSE': rmse, 'R2': r2})

    print(f"{name:<20} | {mae:<10.4f} | {rmse:<10.4f} | {r2:<10.4f}")

# Lưu kết quả ra file Excel/CSV
results_df = pd.DataFrame(results)
results_df.to_csv('../data/processed/baseline_results.csv', index=False)


--- BẢNG KẾT QUẢ BASELINE (MỐC CHUẨN) ---
Model                | MAE        | RMSE       | R2 Score  
------------------------------------------------------------
Linear Regression    | 1.8705     | 4.0201     | 0.4841    
Ridge                | 1.8687     | 4.0165     | 0.4850    
Lasso                | 1.8373     | 3.9974     | 0.4899    
